In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from tensorflow.keras import layers as L
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import Sequence
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import seaborn as sns

# Data Pre-processing

In [ ]:
data = pd.read_csv('../data/age_gender.csv')

## Converting pixels into numpy array
data['pixels']=data['pixels'].apply(lambda x:  np.array(x.split(), dtype="float32"))

data.head()

In [ ]:
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [ ]:
# Prepare the images and resize them to 128x128 using OpenCV
def resize_image(img, size=(224, 224)):
    img = img.reshape((48, 48))  # Reshape to 48x48
    img = cv2.resize(img, size)  # Resize to 128x128
    return img

In [ ]:
X = np.array([resize_image(img) for img in data['pixels']])  # Resize each image
X = np.expand_dims(X, axis=-1)  # Add the channel dimension

In [ ]:
# Normalize the pixel values to [0, 1]
X = X / 255.0

In [ ]:
print('Total rows: {}'.format(len(data)))
print('Total columns: {}'.format(len(data.columns)))

In [ ]:
plt.figure(figsize=(16,16))
for i in range(1500,1520):
    plt.subplot(5,5,(i%25)+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(data['pixels'].iloc[i].reshape(48,48), cmap='gray')
    plt.xlabel(
        "Age:"+str(data['age'].iloc[i])+
        "  Ethnicity:"+str(data['ethnicity'].iloc[i])+
        "  Gender:"+ str(data['gender'].iloc[i])
    )
plt.show()

In [ ]:
y = data['ethnicity']

# Split the data into training and temporary sets
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, random_state=37
)  # 70% training, 30% temporary

# Split the temporary set into validation and test sets
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=37
)  # 50% of 30% = 15% test, 15% validation


# Custom CNN Model for Ethnicity Prediction

In [ ]:
# Define the model
model = tf.keras.Sequential([
    L.InputLayer(input_shape=(224, 224, 1)),
    L.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    L.MaxPooling2D((2, 2)),
    L.Conv2D(64, (3, 3), activation='relu'),
    L.MaxPooling2D((2, 2)),
    L.Flatten(),
    L.Dense(64, activation='relu'),
    L.Dropout(rate=0.5),
    L.Dense(5)  # Output layer with 5 units for 5 ethnicity classes
])

# Print the model summary
model.summary()


In [ ]:
# Compile the model
model.compile(optimizer='rmsprop',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
# Define the callback for saving the best model
checkpoint_callback = ModelCheckpoint(
    'ethnicity_model.keras',  # Path to save the best model
    monitor='val_accuracy',     # Metric to monitor
    save_best_only=True,        # Save only the best model
    mode='max',                 # Maximize the monitored metric
    verbose=1                   # Print messages when saving the model
)

In [ ]:

# Train the model with both callbacks
history = model.fit(
    X_train, y_train,
    epochs=16,
    validation_data=(X_val, y_val),  # Use the separate validation set
    batch_size=64,
    callbacks=[checkpoint_callback]
)

In [ ]:
# Evaluate the model on the test set
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print('Test loss: {}'.format(loss))
print('Test Accuracy: {}'.format(acc))

# Evaluate on test set

In [ ]:
# Function to load model and generate predictions
def load_model_and_predict(model_path, X_test):
    model = tf.keras.models.load_model(model_path)
    y_pred = model.predict(X_test)
    return y_pred


In [ ]:
# Function to plot confusion matrix
def plot_confusion_matrix(y_true, y_pred, labels, title='Confusion Matrix'):
    # Convert probabilities to class labels if necessary
    if len(y_pred.shape) > 1:
        y_pred = np.argmax(y_pred, axis=1)

    cm = confusion_matrix(y_true, y_pred, labels=labels)
    df_cm = pd.DataFrame(cm, index=labels, columns=labels)
    
    plt.figure(figsize=(10, 7))
    sns.heatmap(df_cm, annot=True, fmt='d', cmap='Blues')
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

In [ ]:
# Load the ethnicity model and generate predictions
ethnicity_model_path = '../models/ethnicity_model.keras'
ethnicity_model_predictions = load_model_and_predict(ethnicity_model_path, X_test)

# Convert predictions to class labels
ethnicity_predictions = np.argmax(ethnicity_model_predictions, axis=1)

# Define ethnicity class labels
ethnicity_labels = np.unique(y_test)

In [ ]:
# Plot confusion matrix for ethnicity predictions
plot_confusion_matrix(y_test, ethnicity_predictions, labels=ethnicity_labels, title='Ethnicity Prediction Confusion Matrix')

In [ ]:
# Calculate and print accuracy for ethnicity predictions
ethnicity_accuracy = accuracy_score(y_test, ethnicity_predictions)
print(f"Ethnicity Prediction Accuracy: {ethnicity_accuracy:.4f}")

In [ ]:
# Generate and print classification report
classification_rep = classification_report(y_test, ethnicity_predictions, labels=ethnicity_labels, target_names=[str(label) for label in ethnicity_labels])
print("Classification Report:")
print(classification_rep)